# Data Kung Fu with Pandas

## Import the libraries we'll need.

In [ ]:
import pandas as pd
from pandas.tools.plotting import scatter_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

## Let's read some data!

In [ ]:
raw_dataframe = pd.read_csv("data/mydata.csv")

In [ ]:
raw_dataframe.head(3)

In [ ]:
raw_dataframe.info()

## Let's clean our data.

In [ ]:
dataframe = raw_dataframe.copy()

# Convert _Taxpayer Zip_ to an integer.
dataframe['Taxpayer Zip'] = dataframe['Taxpayer Zip'].fillna(0.0).astype(int)

# Convert _Location County_ to an integer.
dataframe['Location County'] = dataframe['Location County'].fillna(0.0).astype(int)

# Convert _Location Tot Room Receipts_ to a float.
dataframe['Location Tot Room Receipts'] = \
    dataframe['Location Tot Room Receipts'] \
        .replace('[\$,]', '', regex=True).astype(float, inplace=True)

# Convert _Location Tot Room Receipts_ to a float.
dataframe['Location Taxable Receipts'] = \
    dataframe['Location Taxable Receipts'] \
        .replace('[\$,]', '', regex=True).astype(float, inplace=True)

In [ ]:
dataframe.info()

## Exploratory Data Analysis (EDA)

In [ ]:
dataframe.head(3)

In [ ]:
# Hmm, _Location Room Capacity_ looks suspicious
dataframe['Location Room Capacity'].hist(bins=100)

In [ ]:
rows_of_interest = (dataframe['Location Room Capacity'] < 50)
dataframe[rows_of_interest]['Location Room Capacity'].hist(bins=100)

In [ ]:
# How many of these suspicious observations do we have?
(dataframe['Location Room Capacity'] == 1).sum()

<b><font color='indianred'>############################</font><b>

In [ ]:
# IGNORE THIS FOR NOW!
# Keep commented out. We'll come back here later.
#
# print("Orig size:", dataframe.shape)
# dataframe = dataframe[dataframe['Location Room Capacity'] > 1]
# print("New size:", dataframe.shape)

<b><font color='indianred'>############################</font><b>

### Visualization (histograms and scatterplots)

#### Number of States Represented

In [ ]:
# How many states do I got here?
agg = dataframe.groupby('Location State').apply(len)
agg.plot(kind='bar').set_ylabel('# records')

In [ ]:
dataframe['Location State'].value_counts()

In [ ]:
texas_df = dataframe[dataframe['Location State'] == 'TX']

#### Number of Cities Represented
How many unique cities are represented?

In [ ]:
texas_df['Location City'].describe()

#### Histogram: Number of unique hotels in each city

In [ ]:
city_counts = texas_df.groupby('Location City')['Taxpayer Number'].nunique()
city_counts.sort_values(axis=0, ascending=False, inplace=True)
top_cities = city_counts > 50
city_counts[top_cities].plot(kind='bar').set_ylabel('# unique hotels')

#### Exploring the numeric columns

In [ ]:
numeric_columns = ['Month', 'Year', 'Outlet Number', 'Location Room Capacity',
                   'Location Tot Room Receipts', 'Location Taxable Receipts']

In [ ]:
# Let's see some stats
texas_df[numeric_columns].describe()

#### Pairplot over a few numeric columns-of-interest

In [ ]:
cols_of_interest = ['Location Room Capacity', 'Location Tot Room Receipts',
                    'Location Taxable Receipts']
scatter_matrix(texas_df[cols_of_interest], figsize=(10, 8));

#### Compare two cities: Scatterplot 'Location Room Capacity' vs 'Location Tot Room Receipts' for each city

In [ ]:
cities_of_interest = ['AUSTIN', 'EL PASO']

# Scatter plot by city.
for city in cities_of_interest:
    city_mask = (texas_df['Location City'] == city)
    size_mask = (texas_df['Location Room Capacity'] < 220)
    plt.plot(texas_df[city_mask][size_mask]['Location Room Capacity'],
             texas_df[city_mask][size_mask]['Location Tot Room Receipts'],
             linestyle='',
             marker='o',
             alpha=1,
             label=city)
plt.xlabel('Location Room Capacity')
plt.ylabel('Location Tot Room Receipts')
plt.legend(numpoints=1)

#### Histogram of every numeric column

In [ ]:
# Here's another one-liner:
texas_df[numeric_columns].hist(bins=10, figsize=(10, 6))

#### Years and Months, what do we have of each?

In [ ]:
texas_df['Year'].value_counts()

In [ ]:
texas_df['Month'].value_counts()

#### Histogram: Avg 'room capacity' per city (top cities only)

In [ ]:
city_counts = texas_df.groupby('Location City')['Location Room Capacity'].mean()
city_counts.sort_values(axis=0, ascending=False, inplace=True)
city_counts[city_counts>110].plot(kind='bar', figsize=(10, 4)).set_ylabel('avg room capacity')

#### Histogram: Avg income per room per night per city (top cities only)

In [ ]:
num_days = 31 + 31  # <-- both July and August each have 31 days in them.

cities = texas_df.groupby('Location City')
city_receipts = cities['Location Tot Room Receipts'].sum()
city_rooms = cities['Location Room Capacity'].sum()
city_income_per_room_per_night = city_receipts / city_rooms / num_days
city_income_per_room_per_night.sort_values(axis=0, ascending=False, inplace=True)
most_profitable_cities = city_income_per_room_per_night > 100
city_income_per_room_per_night[most_profitable_cities].plot(kind='bar', figsize=(10, 4)) \
    .set_ylabel('avg income per room per night')

#### RED ALERT: What's up with SOUTHLAKE?

In [ ]:
texas_df[texas_df['Location City'] == 'SOUTHLAKE']

Wait. Is this bad data?<br>
Let's check:
http://www3.hilton.com/en/hotels/texas/hilton-dallas-southlake-town-square-DFWSLHF/index.html

#### Is there much of a difference between the month of July and the month of August?

In [ ]:
# How does the specific month affect the avg income per room per night?

cities = texas_df.groupby(['Location City', 'Month'])
city_receipts = cities['Location Tot Room Receipts'].sum()
city_rooms = cities['Location Room Capacity'].sum()
city_income_per_room_per_night = city_receipts / city_rooms / num_days
city_income_per_room_per_night = city_income_per_room_per_night.unstack()
city_income_per_room_per_night['avg_7_8'] = (city_income_per_room_per_night[7] + \
                                             city_income_per_room_per_night[8]) / 2
city_income_per_room_per_night.sort_values(by='avg_7_8', ascending=False, inplace=True)
most_profitable_cities = city_income_per_room_per_night['avg_7_8'] > 100
city_income_per_room_per_night[most_profitable_cities].plot(kind='bar', figsize=(10, 4)) \
    .set_ylabel('avg income per room per night')

In [ ]:
# Seems July (7) was a better month for the top cities than August (8). Let's see if this
# was overall true for the entire dataset.
print('July Avg:', city_income_per_room_per_night[7].mean())
print('Aug Avg: ', city_income_per_room_per_night[8].mean())

####  _Enough for today (for now)_